# Top selling video games in Japan

Coming off from a **quite serious** [first project](https://pmagtulis.github.io/projects.html), I am choosing a rather fun (yet still "editorially sound") project about video games!

**Background:** Famitsu Magazine in Japan releases every year a list of top 100 software games based on the number of units sold. The entire list can be found on Famitsu [website](https://www.famitsu.com/news/202202/06250169.html)-- but it's in Japanese!

So for the purposes of this project, I decided just to google stories that would contain the list in English and collate them.

**Purpose:** My initial vision for this project is just to collate a minimum of 5 years worth of Famitsu rankings into a single dataframe for analysis. However, due to time constraints and some problems gathering data, only years 2019, 2020 and 2021-- covering pre-pandemic and pandemic times-- were included. Some questions I am thinking of getting answers to are:

* How has each game performed through the years?
* Which is the top performing console based on number of games sold?
* How has the pandemic affected game sales?

**Methods:** Since we are not collating the data from a single website, we would have to do a bunch of requests for each site we found. Ideally, one website per one year worth of data.

Once we have all of those, we create a data frame for each and then **concat** everything. We process and analyze and visualize data afterwards.

**Tools:** bs4, pandas, Excel, regex, Adobe Illustrator

# Do all imports

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests import get
import numpy as np
import re

/Users/prinzmagtulis/.pyenv/versions/3.10.0/lib/python3.10/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# Scraping with bs4

Now this is a start of a long process because we would be dealing with different web pages, but let me simplify it in bullet form:

* request and parse each page
* get the list of games 
* make a single data frame for each (ideally, one df per each year)
* clean the data

## Web pages

Below are the list of web pages we scraped and culled information from to get the annual list. As mentioned, the Famitsu list is on Japanese, but is commonly reported by other outlets (i.e. gaming sites, forums, etc.) in English every time it is released.

* 2021 - https://www.nintendolife.com/news/2022/02/japans-top-100-best-selling-video-games-of-2021-according-to-famitsu

* 2020 - https://gamerant.com/japan-best-selling-games-2020-famitsu/

* 2019 - https://twinfinite.net/2020/01/top-100-best-selling-games-2019-japan/

## 2021

In [2]:
raw_html1 = requests.get("https://www.nintendolife.com/news/2022/02/japans-top-100-best-selling-video-games-of-2021-according-to-famitsu").content
doc1 = BeautifulSoup(raw_html1, "html.parser")
#print(doc1.prettify())

In [3]:
container= doc1.find('div', {"class":'body body-text article-text'})
dataset= container.find_all('p')[3]

## Convert to data frame

Using pandas, of course.

In [4]:
df_2021 = pd.DataFrame(dataset)
df_2021

,0
0,"1. [NSW] Monster Hunter Rise – 2,350,693 / NEW"
1,[]
2,\n2. [NSW] Pokemon Brilliant Diamond / Shining...
3,[]
4,"\n3. [NSW] Momotaro Dentetsu: Showa, Heisei, R..."
...,...
194,\n98. [NSW] Moshikashite? Obake no Shatekiya f...
195,[]
196,\n99. [PS5] Marvel’s Spider-Man: Miles Morales...
197,[]


The output is really messy so the next step is to clean it up. 

## Cleaning the data

### Remove the white spaces and NaNs

This will entail **a lot of regex**.

The first task is to remove those seemingly empty rows. This will be followed by removing the white spaces and NaNs. 

The data is not usually this messy as you can strip the text through bs4. But we chose not to do that since we are preserving the numerical order structure of the text.

In [5]:
df_2021 = df_2021.iloc[::2]
df_2021 = df_2021.replace(r'\n',' ', regex=True) 
df_2021 = df_2021.replace(r'\s\s',' ', regex=True) 
df_2021= df_2021.replace(r'^[/d/d]', regex=True)

### Rename the column to 'games'

Until now, we were working with a column with an awful name of '0'.

In [6]:
df_2021.columns = ['games']

### Change the type to string

This is kind of weird to me since a **df1.dtypes** would reveal that the entries are in fact **object** and therefore a string. But using **.strip()** on it doesn't work without us converting the data to string so here we are.

In [7]:
df_2021.games = df_2021.games.astype(str)

In [8]:
df_2021.dtypes

games    object
dtype: object

### Stripping and making new columns

We need to separate the information into new columns and clean up each entries. This should be the final step. And then we request a new webpage and repeat the process until we get the amount of data we need.

In [9]:
df_2021.games.str.strip()                                        #strips out all white spaces
df_2021['console'] = df_2021.games.str.split(expand=True)[1]         #splits the contents and using a part of it in new cols
df_2021['units_sold'] = df_2021.games.str.split(expand=True, pat="–")[1]

In [10]:
df_2021.reset_index(inplace=True)             
df_2021 = df_2021.drop('index', axis=1)                                #reseting the index

### Cleaning your columns of duplicate information

While we stripped the information out of a single column earlier, that original column still contains some of the information we took out of it since stripping does not **transfer** but merely **copy** contents to a new column.

This section cleans everything, including our new columns

In [11]:
df_2021.console = df_2021.console.str.replace("[", "", regex=False)
df_2021.console = df_2021.console.str.replace("]", "", regex=False)
df_2021.console = df_2021.console.str.replace('NS4', 'NSW', regex=False)     #one entry was mistyped as "NS4" instead of "NSW"

#These two clean up the console column of the brackets

In [12]:
df_2021.games = df_2021.games.str.replace(r'[–]\s.+$', "", regex=True)
df_2021.games = df_2021.games.str.replace(r'^\s?\d+[.]\s\s*.{5}\s', "", regex=True)

#This cleans the games column to get only the game name and remove everything else after

In [13]:
df_2021.units_sold = df_2021.units_sold.str.replace(r'[/]\s.+$', "", regex=True)
df_2021.units_sold = df_2021.units_sold.str.replace(',', "", regex=False)

#This cleans the units_sold column and then we convert this to float for analysis later

**IMPORTANT:** Upon checking our regex function for **units_sold** column did not fit two entries because of their long names. We then run a .str.replace for them to put back the figures we lost. 

This is obviously not ideal, but since I'm bad at regex, this will do. :P

In [14]:
df_2021.units_sold = df_2021.units_sold.str.replace('Owaranai Nanokakan no Tabi', "199646", regex=False)
df_2021.units_sold = df_2021.units_sold.str.replace('The Hinokami Chronicles', "150056", regex=False)

df_2021.units_sold = df_2021.units_sold.astype(float)
#Converting the numbers to float format

In [15]:
df_2021['year'] = '2021'
df_2021

,games,console,units_sold,year
0,Monster Hunter Rise,NSW,2350693.0,2021
1,Pokemon Brilliant Diamond / Shining Pearl,NSW,2313115.0,2021
2,"Momotaro Dentetsu: Showa, Heisei, Reiwa mo Tei...",NSW,1266477.0,2021
3,Super Mario 3D World + Bowser’s Fury,NSW,971418.0,2021
4,Ring Fit Adventure,NSW,904685.0,2021
...,...,...,...,...
95,Minecraft Starter Collection,PS4,34389.0,2021
96,Danganronpa Decadence,NSW,34229.0,2021
97,Moshikashite? Obake no Shatekiya for Nintendo ...,NSW,34089.0,2021
98,Marvel’s Spider-Man: Miles Morales,PS5,34016.0,2021


This is just the **first** dataset we need. Below we scrape and clean the 2019 list, from a different website, following the same method.

## 2020

In [16]:
raw_html3 = requests.get('https://gamerant.com/japan-best-selling-games-2020-famitsu/', headers={
"User-Agent" : "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"
}).content
doc3 = BeautifulSoup(raw_html3, "html.parser")
#print(doc3.prettify())

In [17]:
container2= doc3.find('section', {"id": "article-body"})
row= container2.find('ul')
dataset3= row.find_all('li')
#dataset3

In [18]:
df_2020 = pd.DataFrame(dataset3)
df_2020

,0
0,"1. [NSW] Animal Crossing: New Horizons – 6,37..."
1,"2. [NSW] Ring Fit Adventure – 1,591,366 / 2,0..."
2,"3. [NSW] Momotaro Dentetsu: Showa, Heisei, Re..."
3,"4. [PS4] Final Fantasy VII Remake – 949,379 /..."
4,"5. [NSW] Pokemon Sword/Shield – 892,456 / 3,8..."
...,...
95,96. [PS4] Final Fantasy Crystal Chronicles Re...
96,97. [NSW] Kotoba no Puzzle: Moji Pittan Encor...
97,"98. [NSW] Yoshi’s Crafted World – 37,458 / 22..."
98,99. [PS4] NieR Automata Game of the YoRHa Edi...


### Putting up a new column, stripping all white spaces, and 'games' column of separated info

In [19]:
df_2020.columns = ['games']
df_2020.games = df_2020.games.astype(str)
df_2020

,games
0,"1. [NSW] Animal Crossing: New Horizons – 6,37..."
1,"2. [NSW] Ring Fit Adventure – 1,591,366 / 2,0..."
2,"3. [NSW] Momotaro Dentetsu: Showa, Heisei, Re..."
3,"4. [PS4] Final Fantasy VII Remake – 949,379 /..."
4,"5. [NSW] Pokemon Sword/Shield – 892,456 / 3,8..."
...,...
95,96. [PS4] Final Fantasy Crystal Chronicles Re...
96,97. [NSW] Kotoba no Puzzle: Moji Pittan Encor...
97,"98. [NSW] Yoshi’s Crafted World – 37,458 / 22..."
98,99. [PS4] NieR Automata Game of the YoRHa Edi...


In [20]:
df_2020.games.str.strip()                                                 #strips out all white spaces
df_2020['console'] = df_2020.games.str.split(expand=True)[1]               #splits the contents and using a part of it in new cols
df_2020['units_sold'] = df_2020.games.str.split(expand=True, pat="–")[1]

In [21]:
df_2020

,games,console,units_sold
0,"1. [NSW] Animal Crossing: New Horizons – 6,37...",[NSW],"6,378,103 / NEW"
1,"2. [NSW] Ring Fit Adventure – 1,591,366 / 2,0...",[NSW],"1,591,366 / 2,087,005"
2,"3. [NSW] Momotaro Dentetsu: Showa, Heisei, Re...",[NSW],"1,233,023 / NEW"
3,"4. [PS4] Final Fantasy VII Remake – 949,379 /...",[PS4],"949,379 / NEW"
4,"5. [NSW] Pokemon Sword/Shield – 892,456 / 3,8...",[NSW],"892,456 / 3,880,590"
...,...,...,...
95,96. [PS4] Final Fantasy Crystal Chronicles Re...,[PS4],"38,200 / NEW"
96,97. [NSW] Kotoba no Puzzle: Moji Pittan Encor...,[NSW],"38,018 / NEW"
97,"98. [NSW] Yoshi’s Crafted World – 37,458 / 22...",[NSW],"37,458 / 223,523"
98,99. [PS4] NieR Automata Game of the YoRHa Edi...,[PS4],"37,120 / 96,968"


## Cleaning again

Kinda enjoying this part, to be honest. 

### Console column

The **"console"** column needs to be cleaned by removing the brackets.

In [22]:
df_2020.console= df_2020.console.str.strip()
df_2020.console = df_2020.console.str.replace("[", "", regex=False)
df_2020.console = df_2020.console.str.replace("]", "", regex=False)

### Units_sold column

In [23]:
df_2020.units_sold = df_2020.units_sold.str.replace(r'[/]\s.+$', "", regex=True)     #removes all figures after the /
df_2020.units_sold = df_2020.units_sold.str.replace(',', '')

### Games column

In [24]:
df_2020.games = df_2020.games.str.replace(r'[–]\s.+$', "", regex=True)         #strips out all information in games after "-"
df_2020.games = df_2020.games.str.replace(r'[^/d]*[.]', "", regex=True)        #strips out numbers in games column

In [25]:
df_2020

,games,console,units_sold
0,[NSW] Animal Crossing: New Horizons,NSW,6378103
1,[NSW] Ring Fit Adventure,NSW,1591366
2,"[NSW] Momotaro Dentetsu: Showa, Heisei, Reiwa...",NSW,1233023
3,[PS4] Final Fantasy VII Remake,PS4,949379
4,[NSW] Pokemon Sword/Shield,NSW,892456
...,...,...,...
95,[PS4] Final Fantasy Crystal Chronicles Remast...,PS4,38200
96,[NSW] Kotoba no Puzzle: Moji Pittan Encore,NSW,38018
97,[NSW] Yoshi’s Crafted World,NSW,37458
98,[PS4] NieR Automata Game of the YoRHa Edition,PS4,37120


### Transform units_sold column to float and create new year column

In [26]:
df_2020.units_sold= df_2020.units_sold.str.strip()
df_2020.units_sold = df_2020.units_sold.str.replace('Welcome amiibo', "52499", regex=False)
df_2020.units_sold = df_2020.units_sold.str.replace('Waiwai Gakuen Seikatsu', "42077", regex=False)
df_2020.units_sold = df_2020.units_sold.astype(float)

#These strip white spaces, inputs lost information on some entries which did not follow our regex format earlier
#And then convert the column to float.

In [27]:
df_2020['year']= '2020'
df_2020

,games,console,units_sold,year
0,[NSW] Animal Crossing: New Horizons,NSW,6378103.0,2020
1,[NSW] Ring Fit Adventure,NSW,1591366.0,2020
2,"[NSW] Momotaro Dentetsu: Showa, Heisei, Reiwa...",NSW,1233023.0,2020
3,[PS4] Final Fantasy VII Remake,PS4,949379.0,2020
4,[NSW] Pokemon Sword/Shield,NSW,892456.0,2020
...,...,...,...,...
95,[PS4] Final Fantasy Crystal Chronicles Remast...,PS4,38200.0,2020
96,[NSW] Kotoba no Puzzle: Moji Pittan Encore,NSW,38018.0,2020
97,[NSW] Yoshi’s Crafted World,NSW,37458.0,2020
98,[PS4] NieR Automata Game of the YoRHa Edition,PS4,37120.0,2020


## 2019

In [28]:
raw_html2 = requests.get("https://twinfinite.net/2020/01/top-100-best-selling-games-2019-japan/").content
doc2 = BeautifulSoup(raw_html2, "html.parser")
#print(doc2.prettify())

In [29]:
container= doc2.find('ol')
dataset2= container.find_all('li')
#dataset2

## Generate the df and cleaning

This is a repeat of the process above to get a second dataframe, this time with 2019 data in it.

In [30]:
df_2019 = pd.DataFrame(dataset2)
df_2019

,0
0,"Pokemon Sword & Shield – Switch – 2,988,134"
1,"Super Smash Bros. Ultimate – Switch – 1,092,39..."
2,"Super Mario Maker 2 – Switch – 800,504"
3,"Kingdom Hearts 3 – PS4 – 861,226"
4,"New Super Mario Bros. U Deluxe – Switch – 747,589"
...,...
95,"Yakuza 4 – PS4 – 43,782"
96,"Azur Lane: Crosswave – PS4 – 43,585"
97,Tales of Vesperia: Definitive Edition – Switch...
98,"Dead by Daylight – PS4 – 41,682 [59,353]"


### Putting up a new column, stripping all white spaces, and 'games' column of separated info

In [31]:
df_2019.columns = ['games']
df_2019.games = df_2019.games.astype(str)

In [32]:
df_2019.games.str.strip()                                                 #strips out all white spaces
df_2019['console'] = df_2019.games.str.split(expand=True, pat="–")[1]         #splits the contents and using a part of it in new cols
df_2019['units_sold'] = df_2019.games.str.split(expand=True, pat="–")[2]

The **"console"** column is already clean as it is, but we are renaming **Switch** to **NSW** to match its name on the first dataframe.

In [33]:
df_2019.console= df_2019.console.str.strip()
df_2019.console= df_2019.console.str.replace('Switch', 'NSW')

In [34]:
df_2019.games = df_2019.games.str.replace(r'[–]\s.+$', "", regex=True)         #strips out all information in games after "-"

### Cleaning the console column

There were some entries in the console column whose information after the first **"-"**, which we used to remove patterns using regex, contained an extension of their game names, not console names. We manually change this below.

In [35]:
df_2019.console = df_2019.console.str.replace('The Official Video Game', "Switch", regex=False)
df_2019.console = df_2019.console.str.replace('Game of the YoRHa Edition', "PS4", regex=False)
df_2019.console = df_2019.console.str.replace('Welcome amiibo', "3DS", regex=False)
df_2019.console = df_2019.console.str.replace('Ps4', 'PS4', regex=False)              #one entry is named Ps4, instead of PS4

### Cleaning the units_sold column to transform to float

In [36]:
df_2019.units_sold = df_2019.units_sold.str.replace(',', "", regex=False)                #removes all commas
df_2019.units_sold = df_2019.units_sold.str.replace('.', "", regex=False)               #removes a period in one entry
df_2019.units_sold = df_2019.units_sold.str.replace('Switch', "89491", regex=False)     #replaces a name with number  
df_2019.units_sold = df_2019.units_sold.str.replace('PS4', "59848", regex=False)
df_2019.units_sold = df_2019.units_sold.str.replace('3DS', "47880", regex=False)

#These clean the units_sold column and then we convert this to float for analysis later

In [37]:
df_2019.units_sold= df_2019.units_sold.str.strip()
df_2019.units_sold = df_2019.units_sold.str.replace(r'\[\d+\]$', "", regex=True)      #removes all figures from the brackets

In [38]:
df_2019.units_sold= df_2019.units_sold.str.strip()
df_2019.units_sold = df_2019.units_sold.astype(float)

In [39]:
df_2019['year']= '2019'
df_2019

,games,console,units_sold,year
0,Pokemon Sword & Shield,NSW,2988134.0,2019
1,Super Smash Bros. Ultimate,NSW,1092397.0,2019
2,Super Mario Maker 2,NSW,800504.0,2019
3,Kingdom Hearts 3,PS4,861226.0,2019
4,New Super Mario Bros. U Deluxe,NSW,747589.0,2019
...,...,...,...,...
95,Yakuza 4,PS4,43782.0,2019
96,Azur Lane: Crosswave,PS4,43585.0,2019
97,Tales of Vesperia: Definitive Edition,NSW,42816.0,2019
98,Dead by Daylight,PS4,41682.0,2019


### Combine the three dataframes

In [40]:
df_full_list= pd.concat([df_2021, df_2020, df_2019])
df_full_list

# We are not reseting the index because we would want to use it to easily determine the rankings of each game
# per year.

,games,console,units_sold,year
0,Monster Hunter Rise,NSW,2350693.0,2021
1,Pokemon Brilliant Diamond / Shining Pearl,NSW,2313115.0,2021
2,"Momotaro Dentetsu: Showa, Heisei, Reiwa mo Tei...",NSW,1266477.0,2021
3,Super Mario 3D World + Bowser’s Fury,NSW,971418.0,2021
4,Ring Fit Adventure,NSW,904685.0,2021
...,...,...,...,...
95,Yakuza 4,PS4,43782.0,2019
96,Azur Lane: Crosswave,PS4,43585.0,2019
97,Tales of Vesperia: Definitive Edition,NSW,42816.0,2019
98,Dead by Daylight,PS4,41682.0,2019


### Save to CSV

This is for further manual cleaning. Ideally, you would want to do this all in the notebook, but since it's a small dataset, we can do this in **Excel**. We would want to check, for instance, for games with different names in each list but pertaining to the same game.

We will use this CSV for our **initial analysis** below.

In [41]:
#df_full_list.to_csv('video-games-wide.csv')

# Initial analysis

In [42]:
df_full_list

,games,console,units_sold,year
0,Monster Hunter Rise,NSW,2350693.0,2021
1,Pokemon Brilliant Diamond / Shining Pearl,NSW,2313115.0,2021
2,"Momotaro Dentetsu: Showa, Heisei, Reiwa mo Tei...",NSW,1266477.0,2021
3,Super Mario 3D World + Bowser’s Fury,NSW,971418.0,2021
4,Ring Fit Adventure,NSW,904685.0,2021
...,...,...,...,...
95,Yakuza 4,PS4,43782.0,2019
96,Azur Lane: Crosswave,PS4,43585.0,2019
97,Tales of Vesperia: Definitive Edition,NSW,42816.0,2019
98,Dead by Daylight,PS4,41682.0,2019


## How many games from the 2019 list were still in the 2021 list? What were these games?

In [43]:
df_full_list.games.duplicated().value_counts()
#Interpretation: There were 25 games in the 2019 list that still made it in the 2021 list.

False    275
True      25
Name: games, dtype: int64

## What is the average unit sales per each year?

In [59]:
df_full_list.groupby('year').units_sold.mean()

year
2019    203292.06
2020    250093.31
2021    215562.04
Name: units_sold, dtype: float64

## Which year had the most number of game units sold for the Top 100 games?

In [44]:
df_full_list.groupby('year').units_sold.sum()

year
2019    20329206.0
2020    25009331.0
2021    21556204.0
Name: units_sold, dtype: float64

In [45]:
df_full_list.groupby('year').units_sold.sum().pct_change()

year
2019         NaN
2020    0.230217
2021   -0.138074
Name: units_sold, dtype: float64

**Interpretation:** Among those in the list, there was nearly a **14% decrease** in game units sold between 2020 and 2021. That followed a 23% uptick in game units sold from 2019 to 2020.

## How many games were sold by different consoles?

In [46]:
df_pivot = df_full_list.pivot_table(columns="console", index='year', values='units_sold', aggfunc='sum', fill_value='0', dropna=False).reset_index()
df_pivot

console,year,3DS,NSW,PS Vita,PS4,PS5,Switch
0,2019,128189.0,13829419.0,70763.0,6211344.0,0,89491.0
1,2020,52499.0,20042819.0,0,4914013.0,0,0
2,2021,0,19569984.0,0,1735513.0,250707.0,0


In [47]:
df_full_list.groupby('year').console.value_counts()

year  console
2019  NSW        48
      PS4        48
      3DS         2
      PS Vita     1
      Switch      1
2020  NSW        62
      PS4        37
      3DS         1
2021  NSW        75
      PS4        20
      PS5         5
Name: console, dtype: int64

**Interpretation:** Nearly two in three best-selling games in Japan were created by Nintendo in 2020. In 2021, that number rose to three in four.

## By franchise game

## How many units sold were for a Pokemon game? 

In [48]:
df_full_list[df_full_list.games.str.contains(r'\bpokemon', case=False, regex=True)].groupby('year').units_sold.sum()

year
2019    2988134.0
2020    1401962.0
2021    3062072.0
Name: units_sold, dtype: float64

In [49]:
df_full_list[df_full_list.games.str.contains(r'\bpokemon', case=False, regex=True)]

,games,console,units_sold,year
1,Pokemon Brilliant Diamond / Shining Pearl,NSW,2313115.0,2021
10,Pokemon Sword / Shield,NSW,381848.0,2021
13,New Pokemon Snap,NSW,312931.0,2021
72,Pokemon Sword / Shield + Expansion Pass,NSW,54178.0,2021
4,[NSW] Pokemon Sword/Shield,NSW,892456.0,2020
17,[NSW] Pokemon Mystery Dungeon: Rescue Team DX,NSW,279162.0,2020
34,[NSW] Pokemon Sword/Shield + Expansion Pass,NSW,148298.0,2020
53,"[NSW] Pokemon: Let’s Go, Pikachu / Eevee",NSW,82046.0,2020
0,Pokemon Sword & Shield,NSW,2988134.0,2019


## Super Mario?

In [50]:
df_full_list[df_full_list.games.str.contains(r'\bmario', case=False, regex=True)].groupby('year').units_sold.sum()

year
2019    3084642.0
2020    2708614.0
2021    3592691.0
Name: units_sold, dtype: float64

In [51]:
df_full_list[df_full_list.games.str.contains(r'\bmario', case=False, regex=True)]

,games,console,units_sold,year
3,Super Mario 3D World + Bowser’s Fury,NSW,971418.0,2021
5,Mario Kart 8 Deluxe,NSW,815174.0,2021
8,Mario Party Superstars,NSW,628538.0,2021
14,Super Mario Party,NSW,306590.0,2021
24,Mario Golf: Super Rush,NSW,198426.0,2021
32,New Super Mario Bros. U Deluxe,NSW,145871.0,2021
37,Super Mario Maker 2,NSW,139980.0,2021
42,Super Mario Odyssey,NSW,125450.0,2021
43,Super Mario 3D All-Stars,NSW,119330.0,2021
70,Mario & Sonic at the Olympic Games Tokyo 2020,NSW,55321.0,2021


## Resident Evil?

In [52]:
df_full_list[df_full_list.games.str.contains(r'\bresident', case=False, regex=True)]

,games,console,units_sold,year
22,Resident Evil Village,PS4,206312.0,2021
60,Resident Evil Village,PS5,74644.0,2021
19,[PS4] Resident Evil 3,PS4,269187.0,2020
13,Resident Evil 2,PS4,403833.0,2019


In [53]:
df_full_list[df_full_list.games.str.contains(r'\bresident', case=False, regex=True)].groupby('year').units_sold.sum()

year
2019    403833.0
2020    269187.0
2021    280956.0
Name: units_sold, dtype: float64

## Final Fantasy?

In [54]:
df_full_list[df_full_list.games.str.contains(r'\bfinal fantasy', case=False, regex=True)]

,games,console,units_sold,year
99,Final Fantasy VII Remake Intergrade,PS5,34012.0,2021
3,[PS4] Final Fantasy VII Remake,PS4,949379.0,2020
63,[NSW] Final Fantasy Crystal Chronicles Remast...,NSW,64687.0,2020
95,[PS4] Final Fantasy Crystal Chronicles Remast...,PS4,38200.0,2020
79,Final Fantasy X/X-2 HD Remaster,NSW,52670.0,2019
84,Final Fantasy XII The Zodiac Age,NSW,48348.0,2019


In [55]:
df_full_list[df_full_list.games.str.contains(r'\bfinal fantasy', case=False, regex=True)].groupby('year').units_sold.sum()

year
2019     101018.0
2020    1052266.0
2021      34012.0
Name: units_sold, dtype: float64

## Ring Fit Adventure

Not a franchise, but a game on exercise. Look at that number in 2020. 

In [56]:
df_full_list[df_full_list.games.str.contains(r'\bring fit', case=False, regex=True)]

,games,console,units_sold,year
4,Ring Fit Adventure,NSW,904685.0,2021
1,[NSW] Ring Fit Adventure,NSW,1591366.0,2020
9,Ring Fit Adventure,NSW,495639.0,2019


In [57]:
df_full_list[df_full_list.games.str.contains(r'\bring fit', case=False, regex=True)].groupby('year').units_sold.sum().pct_change()

year
2019         NaN
2020    2.210736
2021   -0.431504
Name: units_sold, dtype: float64

**Interpretation:** Units sold spiked by more than three fold in 2020, pandemic year.

That allowed the game to rise in rankings to 2nd spot from 10th the previous year.

# End